# Laplace Equation Solver

## Introduction

In this notebook, we will solve the Laplace equation in two dimensions using an iterative numerical method. We will define various boundary conditions, such as point charge, dipole, and capacitor. This problem is common in electrostatics, where the Laplace equation governs the behavior of electric potential in a region with no charge distribution. The boundary conditions specify how the potential behaves on the edges of the region.

First, let's do the math : 

We aim to develop a script that will numerically evaluate the potential \( V(x, y) \), which is a solution to the Laplace equation:

$$
\nabla^2 V(x, y) = 0
$$

To achieve this, we adopt the following approach:

1. The variables \( x \) and \( y \) will increment by steps \( dx = 1 \) and \( dy = 1 \) instead of varying continuously. This allows us to replace the function \( V(x, y) \) with a matrix \( V[i,j] \), where each element represents the potential at a specific point on the grid.

$$
\begin{pmatrix}
V_{i-1, j-1} & V_{i,j-1} & V_{i+1,j-1} \\
V_{i-1,j} & V_{i,j} & V_{i+1,j} \\
V_{i-1,j+1} & V_{i,j+1} & V_{i+1,j+1}
\end{pmatrix}
$$


The Laplace equation in two dimensions is:

$$
\nabla^2 V(x, y) = \frac{\partial^2 V}{\partial x^2} + \frac{\partial^2 V}{\partial y^2}
$$

We will use Taylor expansions to approximate the second derivatives.




The second-order Taylor expansion of \( V(x_0 + dx, y_0) \) around \( (x_0, y_0) \) is:

$$
V(x_0 + dx, y_0) = V(x_0, y_0) + dx \frac{\partial V}{\partial x} + \frac{dx^2}{2} \frac{\partial^2 V}{\partial x^2} + O(dx^3)
$$

Similarly, the expansion for \( V(x_0 - dx, y_0) \) is:

$$
V(x_0 - dx, y_0) = V(x_0, y_0) - dx \frac{\partial V}{\partial x} + \frac{dx^2}{2} \frac{\partial^2 V}{\partial x^2} + O(dx^3)
$$

Adding these two expansions:

$$
V(x_0 + dx, y_0) + V(x_0 - dx, y_0) = 2V(x_0, y_0) + dx^2 \frac{\partial^2 V}{\partial x^2} + O(dx^3)
$$




Similarly, for the \( y \)-direction:

$$
V(x_0, y_0 + dy) + V(x_0, y_0 - dy) = 2V(x_0, y_0) + dy^2 \frac{\partial^2 V}{\partial y^2} + O(dy^3)
$$




Summing the expansions for both \( x \)- and \( y \)-directions:

$$
V(x_0 + dx, y_0) + V(x_0 - dx, y_0) + V(x_0, y_0 + dy) + V(x_0, y_0 - dy) = 4V(x_0, y_0) + dx^2 \frac{\partial^2 V}{\partial x^2} + dy^2 \frac{\partial^2 V}{\partial y^2}
$$

Since \( dx = dy = 1 \), this simplifies to:

$$
\nabla^2 V(x_0, y_0) \approx V(x_0 + dx, y_0) + V(x_0 - dx, y_0) + V(x_0, y_0 + dy) + V(x_0, y_0 - dy) - 4V(x_0, y_0)
$$

Thus, the discrete form of the Laplace equation is:

$$
V[i,j] = \frac{V[i-1,j] + V[i+1,j] + V[i,j-1] + V[i,j+1]}{4}
$$


Then, let's break down the code !

## Libraries
We begin by importing the necessary libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import *

## Constants
Define the constants for the problem. \( V_0 \) is the potential on the boundary, and \( V_1 \) is the potential at specific points.


In [ ]:
V0 = 0
V1 = 200

## Functions to Set Boundary Conditions


We will define functions to create various electrostatic configurations. These will modify the boundary conditions in the potential matrix.


### Point Charge
The `point_charge` function places a single point charge in the center of the grid.


In [ ]:
def point_charge(B, tab):
    B[N//2, N//2] = 1
    tab[N//2, N//2] = V1

### Dipole
The `dipole` function creates a dipole by placing two opposite charges in the center of the grid.

In [ ]:
def dipole(B, t):
    B[N//2, N//2 + N//20] = 1
    B[N//2, N//2 - N//20] = 1
    t[N//2, N//2 + N//20] = -V1
    t[N//2, N//2 - N//20] = V1


### Capacitor
The `capacitor` function simulates a parallel plate capacitor by setting the potential at two parallel lines.

In [ ]:
def capacitor(B, t):
    for j in range(N//2 - N//10, N//2 + N//10):
        B[j, N//2 + N//20] = 1
        B[j, N//2 - N//20] = 1
        t[j, N//2 + N//20] = -V1
        t[j, N//2 - N//20] = V1

### Tip Effect
This function sets up a "tip effect" or "corona discharge" scenario, where there is a sharp point with varying potential across the boundaries.

In [ ]:
def tip_effect(B, V):
    Vmin, Vmax = 0, 10 * V1
    h, d = 15, 1
    
    # Bottom and top borders:
    B[0, :], B[N-1, :] = 1, 1
    V[0, :], V[N-1, :] = Vmin, Vmax

    # Left border:
    B[:, 0] = 1
    V[:, 0] = np.linspace(Vmin, Vmax, N)

    # Right border:
    B[:, N-1] = 1
    V[:, N-1] = np.linspace(Vmin, Vmax, N)
    
    # Tip:
    B[0:h, N//2 - d//2:N//2 + (d+1)//2] = 1
    V[0:h, N//2 - d//2:N//2 + (d+1)//2] = Vmin
    x, y = h, N//2
    for k in range(N):
        for i in range(N):
            if (k - x)**2 + (i - y)**2 <= (d//2)**2:
                B[k, i] = 1
                V[k, i] = Vmin


## Initialization of the Grid

The `init_grid` function initializes the grid by setting the boundary conditions using the `tip_effect` method.


In [ ]:
def init_grid():
    B = np.zeros((N, N))
    tab = np.zeros((N, N))
    tip_effect(B, tab)
    return tab, B

## Iterative Solver

This function applies the iterative method to solve the Laplace equation, updating the potential based on the average of the neighboring values.

In [ ]:
def iterate(tab):
    ecart = 0
    temp = tab.copy()
    for i in range(1, N-1):
        for j in range(1, N-1):
            if B[i, j] == 0:
                temp[i, j] = (tab[i-1, j] + tab[i+1, j] + tab[i, j-1] + tab[i, j+1]) / 4
            ecart = max(ecart, abs(temp[i, j] - tab[i, j]))
    return temp, ecart


## Solution Process

We now set up the grid and apply the iterative solver until the change in potential between iterations is smaller than a chosen tolerance `eps`.


In [ ]:
N = 60  # Grid size
eps = 0.3  # Tolerance for convergence

pot, B = init_grid()

while True:
    pot, e = iterate(pot)
    if e < eps:
        break

## Visualization

Finally, we visualize the results using a contour plot of the potential and display the boundary conditions as a binary image.


In [ ]:
def plot_potential(B, f, color_map=False, n_contours=25):
    N, _ = B.shape
    plt.figure()
    plt.imshow(B, origin='lower', cmap='binary', interpolation='nearest')
    if color_map:
        plt.imshow(f, origin='lower')
        plt.colorbar()
    x = np.linspace(0, N-1, N)
    y = np.linspace(0, N-1, N)
    contours = plt.contour(y, x, f, n_contours, colors='k')
    plt.clabel(contours, fmt='%1.1f')
    plt.show()

plot_potential(B, pot)

## Conclusion

This notebook presents a simple method for solving the Laplace equation numerically in two dimensions. The potential is computed iteratively based on the boundary conditions, and the results are visualized using contour plots to display the equipotential lines.